In [1]:
import pretty_midi
import glob
import pickle

from keras.layers import Lambda, Input, Dense
from keras.models import Model
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import os
import matplotlib.cm as cm

Using TensorFlow backend.


In [2]:
def get_filename(filepath):
    base = os.path.basename(filepath)
    return os.path.splitext(base)[0]

In [3]:
NPITCH = 128
# output_t_size = 500 = 5 * 100 bc. 100 is default freq pretty_midi
def export_piano_rolls(roll, song_filename, output_t_size=500):
    pitch, ntime = roll.shape
    if pitch != NPITCH:
        print("Mismatch number of pitches")
    
    # Step half of size to train a connection between two pieces
    for st in range(0, ntime, output_t_size//2):
        roll_img = roll[:,st:st+output_t_size]
        if st + output_t_size > ntime:
            break
        dirpath = 'trainings/roll_imgs_velo/{}/'.format(song_filename)
        if st == 0 and not os.path.exists(dirpath):
            os.makedirs(dirpath)
        plt.imsave('{}st-{}.png'.format(dirpath, st), roll_img, cmap=cm.gray)

In [6]:
def get_piano_rolls():
    pr = []
    for file in glob.glob("../type0/*.midi"):
        pm = pretty_midi.PrettyMIDI(file) #takes a midi file and converts to pretty_midi
        instr = pm.instruments #splits into list of instruments
        print("Processing ", file)
        for instrument in instr:
            name = pretty_midi.program_to_instrument_name(instrument.program)
            if name == "Acoustic Grand Piano": #only take the piano track
                piano_roll = instrument.get_piano_roll() #get the piano roll, which is a np.ndarray
                pr.append(piano_roll)  
                roll = piano_roll
#                 roll = (piano_roll[:,:]>0).astype(int)
                filename = get_filename(file)
                export_piano_rolls(roll, filename, output_t_size=128)
    return pr #list of piano rolls (np.ndarray matrices)

In [7]:
get_piano_rolls()

Processing  ../type0/beethoven_opus22_2_format0.midi
Processing  ../type0/chpn-p21_format0.midi
Processing  ../type0/mendel_op19_1_format0.midi
Processing  ../type0/mendel_op19_6_format0.midi
Processing  ../type0/liz_et1_format0.midi
Processing  ../type0/burg_spinnerlied_format0.midi
Processing  ../type0/liz_et6_format0.midi
Processing  ../type0/pathetique_1_format0.midi
Processing  ../type0/clementi_opus36_3_3_format0.midi
Processing  ../type0/clementi_opus36_5_2_format0.midi
Processing  ../type0/rac_op33_6_format0.midi
Processing  ../type0/ty_september_format0.midi
Processing  ../type0/mz_331_1_format0.midi
Processing  ../type0/mz_311_2_format0.midi
Processing  ../type0/beethoven_opus10_1_format0.midi
Processing  ../type0/burg_erwachen_format0.midi
Processing  ../type0/chpn-p10_format0.midi
Processing  ../type0/scn15_1_format0.midi
Processing  ../type0/mendel_op30_4_format0.midi
Processing  ../type0/schubert_D850_4_format0.midi
Processing  ../type0/chpn-p17_format0.midi
Processing  .

Processing  ../type0/chpn-p3_format0.midi
Processing  ../type0/mond_1_format0.midi
Processing  ../type0/scn16_5_format0.midi
Processing  ../type0/burg_perlen_format0.midi
Processing  ../type0/mendel_op62_3_format0.midi
Processing  ../type0/alb_se4_format0.midi
Processing  ../type0/haydn_33_2_format0.midi
Processing  ../type0/clementi_opus36_1_1_format0.midi
Processing  ../type0/chpn_op53_format0.midi
Processing  ../type0/bor_ps3_format0.midi
Processing  ../type0/ty_juli_format0.midi
Processing  ../type0/brahms_opus1_2_format0.midi
Processing  ../type0/alb_se3_format0.midi
Processing  ../type0/liz_et_trans5_format0.midi
Processing  ../type0/haydn_35_3_format0.midi
Processing  ../type0/debussy_cc_1_format0.midi
Processing  ../type0/chpn_op35_2_format0.midi
Processing  ../type0/br_rhap_format0.midi
Processing  ../type0/debussy_cc_6_format0.midi
Processing  ../type0/haydn_9_1_format0.midi
Processing  ../type0/chpn_op33_4_format0.midi
Processing  ../type0/haydn_43_2_format0.midi
Processing 

[array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]), array([